
# Основы Python для биоинформатики
**База для ноутбука с графами, DFS, OLC и топологической сортировкой**

Этот ноутбук коротко покрывает всё, что нужно для выполнения заданий:
- переменные и типы,
- строки и срезы,
- списки, словари и множества,
- циклы и условия,
- функции,
- вложенные циклы для поиска перекрытий,
- минимальные заготовки: стековый DFS и топологическая сортировка (Kahn).

> Запускайте ячейки по порядку. Задания помечены как **Задание**. 



## 1. Переменные и типы данных
Основные типы:
- `int`, `float` — числа,
- `str` — строки,
- `list` — списки,
- `dict` — словари,
- `set` — множества,
- `bool` — логический тип (`True/False`).


In [ ]:

x = 5              # int
pi = 3.14          # float
seq = "ATCG"       # str
reads = ["ATCG", "TCGA"]  # list
overlaps = {"ATCG": ["TCGA"]}  # dict
unique_bases = {"A", "T", "C", "G"}  # set
flag = True        # bool

print(type(x), type(pi), type(seq), type(reads), type(overlaps), type(unique_bases), type(flag))



## 2. Строки: индексация и срезы
Строки важны для работы с ДНК:
- индексы с 0,
- срезы — `s[a:b]`, `s[:k]`, `s[-k:]`,
- операции: конкатенация, поиск подстроки.


In [ ]:

seq = "ATTACG"

print("Первый символ:", seq[0])
print("Последние три:", seq[-3:])
print("Без первых двух:", seq[2:])
print("Объединение:", seq + "TACG")
print("Длина:", len(seq))
print("'TAC' in seq?", "TAC" in seq)



## 3. Условия (`if / elif / else`) и логика
Пример: посчитаем GC-контент.


In [ ]:

seq = "ATCGGCGC"

gc = 0
for base in seq:
    if base in ["G", "C"]:
        gc += 1

gc_content = gc / len(seq)
print("GC-контент:", round(gc_content * 100, 2), "%")



## 4. Списки и циклы
Списки — упорядоченные коллекции. По ним удобно итерироваться.


In [ ]:

reads = ["ATTAC", "TACG", "CGT"]
print("Первый элемент:", reads[0])
reads.append("ACG")
print("После добавления:", reads)

print("Перебор:")
for r in reads:
    print("  read:", r)



## 5. Функции
Функции помогают переиспользовать код и структурировать решения.


In [ ]:

def gc_content(seq: str) -> float:
    """Возвращает долю G и C в последовательности."""
    gc = sum(1 for b in seq if b in "GC")
    return gc / len(seq)

print(gc_content("ATCG"))
print(gc_content("GGGGCCCC"))



## 6. Словари (`dict`) — представление графов
Словарь можно использовать как граф: ключ — вершина, значение — список соседей.


In [ ]:

graph = {"A": ["B", "C"], "B": ["D"], "C": [], "D": []}
print("Соседи A:", graph["A"])

# Добавим новую вершину
graph["E"] = ["A"]
print("Теперь есть вершина E:", graph)



## 7. Множества (`set`) — учёт посещённых вершин
Множество не хранит дубликаты и проверяет принадлежность за амортизированное O(1).


In [ ]:

visited = set()
visited.add("A")
visited.add("B")
visited.add("A")  # дубликат не добавится
print("visited =", visited)
print("'A' in visited?", "A" in visited)



## 8. Вложенные циклы — поиск перекрытий
Частая операция: сравнить каждую строку с каждой.


In [ ]:

reads = ["ATTAC", "TACG", "CGT"]
k = 2  # длина перекрытия
pairs = []
for a in reads:
    for b in reads:
        if a != b and a[-k:] == b[:k]:
            pairs.append((a, b))
pairs



### 🧪 Задание 1
Напишите функцию `count_kmer(seq, kmer)`, которая считает, сколько раз подстрока `kmer` встречается в `seq` (с перекрытиями).


In [ ]:

# TODO: реализуйте count_kmer(seq, kmer)
def count_kmer(seq: str, kmer: str) -> int:
    count = 0
    # ваш код здесь
    for i in range(len(seq) - len(kmer) + 1):
        if seq[i:i+len(kmer)] == kmer:
            count += 1
    return count

# проверка
assert count_kmer("ATATAT", "ATA") == 2
assert count_kmer("AAAAA", "AA") == 4
print("OK")



### Задание 2: Итеративный DFS (без рекурсии)
**Идея.** DFS уходит «в глубину», а стек как раз хранит «последним вошёл — первым вышел».  
Алгоритм (итеративно):
1) Положить стартовую вершину в стек.  
2) Пока стек не пуст:  
   — достать вершину;  
   — если ещё не посещали → пометить, записать в порядок;  
   — положить в стек её соседей (так, чтобы следующий шаг шёл «вглубь»).  

> Замечание: порядок обхода зависит от порядка соседей и того, в каком порядке вы кладёте их в стек. Допустимо любое последовательное правило, но **всегда** не заходите повторно в уже посещённые вершины.


In [ ]:
#  Реализуйте итеративный DFS.
# Требования:
# - функция возвращает список вершин в порядке посещения;
# - использует только цикл и стек (список), без рекурсии;
# - не зацикливается на графах с циклами (используйте visited);
# - не модифицирует исходный граф;
# - если start нет в графе — верните [].

def dfs_iter(graph: dict, start) -> list:
    """
    Итеративный обход в глубину.
    graph: {node: [neighbors, ...]}
    start: стартовая вершина (должна быть ключом в graph)
    """
    # TODO: напишите реализацию
    visited = set()
    order = []
    if start not in graph:
        return order

    stack = [start]
    while stack:
        node = stack.pop()
        if node in visited:
            continue
        visited.add(node)
        order.append(node)

        # TODO: положите соседей node в стек.
        # Подсказка: чтобы получить порядок ближе к «рекурсивному»,
        # можно итерироваться по соседям в обратном порядке.
        for neigh in reversed(graph.get(node, [])):
            if neigh not in visited:
                stack.append(neigh)

    return order


In [ ]:
def check_reachability(graph, start, order):
    # проверяем, что это список уникальных вершин, достижимых из start
    reachable = set()
    # простая очередная волна для достижимости (BFS), только для проверки
    from collections import deque
    dq = deque([start]) if start in graph else deque()
    while dq:
        u = dq.popleft()
        if u in reachable:
            continue
        reachable.add(u)
        for v in graph.get(u, []):
            if v not in reachable:
                dq.append(v)
    return set(order) == reachable and len(order) == len(reachable)

In [ ]:
# Тест 1: простой ацикличный граф
g1 = {
    "A": ["B", "C"],
    "B": ["D"],
    "C": [],
    "D": []
}
ord1 = dfs_iter(g1, "A")
assert ord1[0] == "A"
assert check_reachability(g1, "A", ord1)

# Тест 2: граф с циклом (A -> B -> C -> A), не должно зависнуть
g2 = {
    "A": ["B"],
    "B": ["C"],
    "C": ["A"],
}
ord2 = dfs_iter(g2, "A")
assert ord2[0] == "A"
assert check_reachability(g2, "A", ord2)

# Тест 3: несвязный граф (старт в отдельной компоненте)
g3 = {
    "S": ["X"],
    "X": [],
    "P": ["Q"],  # другая компонента
    "Q": [],
}
ord3 = dfs_iter(g3, "S")
assert set(ord3) == {"S", "X"}
assert check_reachability(g3, "S", ord3)

# Тест 4: стартовой вершины нет
ord4 = dfs_iter(g1, "Z")
assert ord4 == []

print("Все проверки пройдены ✅")